# Preparing our Data for Fine-Tuning / or Text Generation / Stimuli
This file covers how a function was created that could shorten articles based on the requirements mentioned in the thesis. It also includes how these articles then could be converted into the JSONL format which was necessary for fine-tuning GPT-3

In [2]:
from nltk.tokenize import word_tokenize, TreebankWordDetokenizer
import pandas as pd

## Functions to shorten articles

In [2]:
def shorten_article(article: str) -> str:
    '''
    Function to shorten an article to roughly 120 words using nltk (stops at the first natural punctuation after 120 words) 

    Parameters
    article (text to be shortened): str
    '''

    tokens = word_tokenize(article) #use nltk to tokenize the text
    punctuations = [".", "!", "?", ","]
    stops = [".", "!", "?",]

    counter = 0 
    
    short_article = []
    
    for i in range(len(tokens)): 
        if counter > 120 and tokens[i] in stops: # if counter is over 120 & token is a stop, break the loop (to not cut in middle of a sentence)
            short_article.append(tokens[i]) 
            break
        else: 
            if tokens[i] not in punctuations: # count the number of words (not punctuations)
                short_article.append(tokens[i])
                counter += 1
            else: 
                short_article.append(tokens[i]) # add punctuation to the list but not count it

    detokenize_article = TreebankWordDetokenizer().detokenize(short_article) # use nltk to detokenize the text
    detokenize_article = detokenize_article.replace(" .", ".") # fix detokenization errors

    return detokenize_article

def shorten_all_articles(text_column:pd.Series)-> list:
    '''
    Function to shorten all articles in a text column using the shorten_article function

    Parameters
    text_column (column with articles to be shortened): pd.Series
    '''
    short_articles = []
    for article in text_column:
        short_articles.append(shorten_article(article))
    return short_articles

## Function to convert short articles to JSONL

In [3]:
def jsonl_article_data(data, header_col, sub_header_col, short_text_col, output_name):
    ''' 
    Function to create a jsonl file with the following structure:
    prompt: <header> <subheader>
    completion: <short_text>

    Parameters:
    data (pandas dataframe): dataframe with the data
    header_col (str): name of the column with the header
    sub_header_col (str): name of the column with the subheader
    short_text_col (str): name of the column with the shortened text
    output_name (str): name of the output file
    '''

    # create prompt column
    data["prompt"] = data[header_col].astype(str) +". "+ data[sub_header_col]

    # rename short_text to completion
    data = data.rename(columns={short_text_col: "completion"})

    # rearrange columns for jsonl format
    data = data[["prompt", "completion"]]


    # prepare for jsonl fine-tuning (prompts ending with "->" and completitons starting with white space " " & ending with "\n")
    data["prompt"] = data["prompt"].astype(str) + " ->"
    data["completion"] = " "+ data["completion"].astype(str) + " \n"

    short_json = data.to_json(f"{output_name}.jsonl", orient='records', lines=True, force_ascii=False) #needs force_ascii=False to keep danish characters

## Shuffling Articles

In [5]:
data = pd.read_csv("~/Desktop/politics-GPT3/data/articles (20. sep - 30. oct)/final_articles (20. sep - 30. oct).csv")

In [8]:
data["short_text"].head(10)

0    En ny geopolitisk orden i Europa. Sammenstød m...
1    Græsset står højt ved Per Lauritsens hus i Aal...
2    Elena Smith er 71 år og aktivist. Da hun var y...
3    Jesper Jarlbæk viser rundt på Christianshavn, ...
4    Tidligere summede burene i de store haller af ...
5    Da den tidligere ejer af Hesbjerg Slot Jørgen ...
6    Der venter en historisk stor begivenhed, når v...
7    En kvinde i 60'erne er søndag formiddag afgået...
8    I 2023 har 19 kommuner fået mulighed for enten...
9    Niels Laursen kan se spidsen af ti vindmøller ...
Name: short_text, dtype: object

In [11]:
data = data.sample(frac=1)

In [13]:
data.to_csv("shuffled_final_articles (20. sep - 30. oct).csv")

## Using the functions on our data 

In [ ]:
samf_data = pd.read_csv("data/nyheder_samfund_only.csv")

In [ ]:
samf_data["short_text"] = shorten_all_articles(samf_data['tekst'])

In [ ]:
samf_data["short_text"][0]

'Titusindvis af ansøgere får snart besked om, hvilken videregående uddannelse de er kommet ind på - og dermed også om deres karaktergennemsnit er over adgangskvotienten. Men mange kommer ikke ind på en af deres ønskede uddannelser og skal i stedet kigge sig om efter andre muligheder. Til dem (og andre nysgerrige) har vi lave denne side, der kan fortælle, hvor mange og hvilke uddannelser snittet rækker til - hvis man ellers opfylder de andre krav til for eksempel særlige fag. Træk i håndtaget for at se, hvor mange uddannelser dit karaktergennemsnit giver adgang til:, Er dit eksamensbevis med karakterer fra den gamle 13-skala, kan du se her, hvordan du omregner den til den nye syvtalsskala. Listen rummer kun de videregående uddannelser, der er en del af Den Koordinerede Tilmelding (KOT).'

In [ ]:
jsonl_article_data(samf_data, "headers", "sub_header", "short_text", "samf_articles")